# Custom model deployment using Sagemaker

In [1]:
# install sagemaker
import sys
!{sys.executable} -m pip install sagemaker==1.72

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
# importing the required libraries
import os
import sys
import json
import time
import io
import boto3
from time import strftime, gmtime

from IPython.display import display
import re
import sagemaker
from sagemaker import get_execution_role

In [3]:
# grab the sagemaker client, boto3 session and execution roles to create a bucket to store the model outputs
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

In [4]:
# giving the docker image name
docker_image_name = "<Replace this with your ECR repository registory link"

In [5]:
# s3 bucket storing the model output data is mentioned below
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()['Account']
bucket = "sagemaker-studio-{}/{}".format(sess.region_name, account_id)
prefix = "fraud-detection"

## Creating estimator with custom model

In [6]:
sess = sagemaker.session.Session()
fraud_detector = sagemaker.estimator.Estimator(image_name=docker_image_name,
                                              role=role, train_instance_count=1,
                                              train_instance_type='ml.m5.xlarge',
                                              output_path = "s3://{}/{}/output".format(bucket, prefix),
                                              base_job_name="fraud-detection",
                                              sagemaker_session=sess)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


## Train the model on the training data located in the container image

In [ ]:
fraud_detector.fit()

## Deploy the custom trained model

In [8]:
endpoint_name = "fraud-detection-endpoint"

In [ ]:
fraud_detector.deploy(initial_instance_count=1, 
                    instance_type='ml.m5.xlarge', 
                    endpoint_name=endpoint_name)

In [ ]:
# deleting the endpoint when we are done with the inference
sm.delete_endpoint(EndpointName=endpoint_name)